# Introdução
Breve introdução sobre o projeto e sobre a etapa de pré-processamento.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-4b7a7198-9051-5722-6165-d92df98b2293)


In [ ]:
#use this command to see GPU activity while doing Deep Learning tasks, for this command 'nvidia-smi' and for above one to work, go to 'Runtime > change runtime type > Hardware Accelerator > GPU'
!nvidia-smi

Sun Sep 18 18:37:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    34W / 250W |   4579MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install translators --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 76 kB 6.2 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 115 kB 53.6 MB/s 
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14598 sha256=e58ad27455892b4fd10a8254fb122a376967757f3f056cc23a0e80635694f288
  Stored in directory: /root/.cache/pip/wheels/9a/ee/03/da5c0b4a8c13362beeb844eb913bbe58a89bde1de2b9157007
Successfully built PyExecJS
ERROR: Operation cancelled by user


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 8.1 MB/s 
     |████████████████████████████████| 287 kB 64.1 MB/s 


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 2.1 MB/s 
     |████████████████████████████████| 6.6 MB 33.0 MB/s 
     |████████████████████████████████| 120 kB 38.6 MB/s 


In [ ]:
import math
import time
import requests
#import contractions
import unicodedata
# import translators as ts

import numpy as np
from numpy import array
import tensorflow as tf 
from tensorflow import keras

import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import Activation
from keras.layers import GlobalMaxPool1D
from keras.layers import Dropout, Conv1D, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig, TFLongformerModel, TFLongformerForSequenceClassification
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import os

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
import json

def write_list(a_list, file_name="names.json"):
    print("Started writing list data into a json file")
    with open(file_name, "w") as fp:
        json.dump(a_list, fp)
        print("Done writing JSON data into .json file")

# Read list to memory
def read_list(file_name="names.json"):
    # for reading also binary mode is important
    with open(file_name, 'rb') as fp:
        n_list = json.load(fp)
        return n_list

# assume you have the following list
names = ['Jessa', 'Eric', 'Bob']
write_list(names)
r_names = read_list()
print('List is', r_names)

Started writing list data into a json file
Done writing JSON data into .json file
List is ['Jessa', 'Eric', 'Bob']


In [ ]:
xp8_sentences = read_list("/content/drive/MyDrive/Mestrado/PesosKDeep/xp8_sent.json")
print(xp8_sentences[450])

opportunity technical trs training vacancy trs ttiii fapesp scholarship young researcher immodulation project iron homeostasis regularity signification tyrosine quinase tam infection mycobacterium tuberculosis target development hosttargete immunopharmacological therapy develop department bioquica immunology ribeiro preto medical school university paulo fmrpusp coordination dr diego lus costa select candidate assist maintenance genotyping transgenic mouse colnie reagent consumable monitoring monitoring technician support laboratory uspbrmore information requirement benefit fapesp ttiii scholarship fapespbr fapespbr


In [ ]:
sentencesMCTIList_xp8 = read_list("/content/drive/MyDrive/Mestrado/PesosKDeep/xp8_list.json")
print(sentencesMCTIList_xp8[450])

['opportunity', 'technical', 'trs', 'training', 'vacancy', 'trs', 'ttiii', 'fapesp', 'scholarship', 'young', 'researcher', 'immodulation', 'project', 'iron', 'homeostasis', 'regularity', 'signification', 'tyrosine', 'quinase', 'tam', 'infection', 'mycobacterium', 'tuberculosis', 'target', 'development', 'hosttargete', 'immunopharmacological', 'therapy', 'develop', 'department', 'bioquica', 'immunology', 'ribeiro', 'preto', 'medical', 'school', 'university', 'paulo', 'fmrpusp', 'coordination', 'dr', 'diego', 'lus', 'costa', 'select', 'candidate', 'assist', 'maintenance', 'genotyping', 'transgenic', 'mouse', 'colnie', 'reagent', 'consumable', 'monitoring', 'monitoring', 'technician', 'support', 'laboratory', 'uspbrmore', 'information', 'requirement', 'benefit', 'fapesp', 'ttiii', 'scholarship', 'fapespbr', 'fapespbr']


In [ ]:
max_len = 0
count_4096 = 0
for sent in sentencesMCTIList_xp8:
  if len(sent) > max_len:
    max_len = len(sent)
  if len(sent) > 2048:
    count_4096+=1
print(max_len)
print(count_4096)

2726
9


In [ ]:
model = TFLongformerModel.from_pretrained('allenai/longformer-base-4096',
                                                          gradient_checkpointing=False,
                                                          attention_window = 512)

tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096', max_length = 4096)

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/765M [00:00<?, ?B/s]

Some layers from the model checkpoint at allenai/longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at allenai/longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
teste = "hello my friend"
tkn = tokenizer(teste, return_tensors="tf",  padding = 'max_length', truncation=True)
print(tkn.input_ids)
np.shape(model(tkn.input_ids)[0][0])

tf.Tensor([[    0 42891   127 ...     1     1     1]], shape=(1, 4096), dtype=int32)


TensorShape([4096, 768])

In [ ]:
encodings = []
for sentence in xp8_sentences:
  encoding = tokenizer(sentence, return_tensors="tf",  padding = 'max_length', truncation=True)
  encodings.append(encoding)

In [ ]:
model_outputs = []
for encoding in encodings:
  input_ids = encoding["input_ids"]
  output_without_mask = model(input_ids)[0]
  model_outputs.append(output_without_mask.numpy())

model_outputs = np.array(model_outputs)

In [ ]:
np.save("/content/drive/MyDrive/Mestrado/PesosLDeep/longformer_model_short.npy", model_outputs)

In [ ]:
model_outputs = np.load("/content/drive/MyDrive/Mestrado/PesosLDeep/longformer_model.npy")

In [ ]:
print(model_outputs.shape)

(928, 1, 4096, 768)


In [ ]:
print(np.shape(model_outputs[0]))

(1, 4096, 768)


Dataset

In [ ]:
url= 'https://github.com/chap0lin/PPF-MCTI/blob/master/Datasets/oportunidadesrotulo%20-%20%C3%9Altima.xlsx?raw=true'
myfile = requests.get(url)
# dataMCTI = pd.read_excel(myfile.content, sheet_name='oportunidadesrotulo')
dataMCTI = pd.read_excel(myfile.content)


print("MCTI Dataset has " + str(dataMCTI.shape[0]) + " examples with " + str(dataMCTI.shape[1]) + " columns of information")

MCTI Dataset has 928 examples with 21 columns of information


In [ ]:
labels = dataMCTI['opo_brazil']
labelsMCTI = np.where(labels == "Y", 1, 0)

In [ ]:
model_outputs = np.reshape(model_outputs, (928,4096,768))
model_outputs.shape

(928, 4096, 768)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(model_outputs, labelsMCTI, test_size=0.20, random_state=20)

In [ ]:
print(np.shape(x_train[0]))
print(np.shape(model_outputs[0]))

(4096, 768)
(4096, 768)


In [ ]:
del model_outputs

In [ ]:
def train_network_nn(save_as='best weights.h5'):
  
  # Creating the Network
  model_NN = Sequential() 
  model_NN.add(Input(shape=np.shape(x_train[0])))
  model_NN.add(Flatten())
  model_NN.add(Dense(1, activation='relu'))

  # add checkpoint to save the network and stop if training doesn't improve MCTI
  checkpoint = keras.callbacks.ModelCheckpoint(save_as, monitor='val_accuracy', verbose=1, 
                              save_best_only=True, mode='max')
  callbacks_list = [checkpoint]

  model_NN.compile(optimizer='adam', loss='binary_crossentropy', 
                        metrics=['accuracy', f1_m, precision_m, recall_m])

  # Fiting  the model
  history = model_NN.fit(x_train, y_train, epochs=100, 
                        callbacks=callbacks_list, verbose=2, 
                        validation_data=(x_test, y_test), 
                        batch_size=16)

  # Evaluate the model
  [modelloss, modelaccuracy, 
  modelf1, modelprecision, 
  modelrecall] = model_NN.evaluate(x_test, y_test, verbose=0)

  #plot_history(history)
  return modelaccuracy, (x_test, y_test)


In [ ]:
def train_network_dnn(save_as='best weights.h5'):
  
  # Creating the Network
  model_DNN = Sequential()
  model_DNN.add(Dense(512, input_shape=np.shape(x_train[0])))
  model_DNN.add(Activation('relu'))
  model_DNN.add(Dense(256))
  model_DNN.add(Activation('relu'))
  model_DNN.add(Dense(128))
  model_DNN.add(Activation('relu'))
  model_DNN.add(Flatten())
  model_DNN.add(Dense(1, activation='sigmoid'))

  # add checkpoint to save the network and stop if training doesn't improve MCTI
  checkpoint = keras.callbacks.ModelCheckpoint(save_as, monitor='val_accuracy', verbose=1, 
                              save_best_only=True, mode='max')
  callbacks_list = [checkpoint]

  model_DNN.compile(optimizer='adam', loss='binary_crossentropy', 
                        metrics=['accuracy', f1_m, precision_m, recall_m])

  # Fiting  the model
  history = model_DNN.fit(x_train, y_train, epochs=100, 
                        callbacks=callbacks_list, verbose=2, 
                        validation_data=(x_test, y_test), 
                        batch_size=8)

  # Evaluate the model
  [modelloss, modelaccuracy, 
  modelf1, modelprecision, 
  modelrecall] = model_DNN.evaluate(x_test, y_test, verbose=0)

  #plot_history(history)
  return modelaccuracy, (x_test, y_test)


In [ ]:
def train_network_cnn(save_as='best weights.h5'):
  
  # Creating the Network
  model_CNN = Sequential()
  model_CNN.add(Input(shape=np.shape(x_train[0])))
  model_CNN.add(Dropout(0.5))
  model_CNN.add(Conv1D(128, 7, padding="valid", activation="relu", strides=3))
  model_CNN.add(Conv1D(128, 7, padding="valid", activation="relu", strides=3))
  model_CNN.add(GlobalMaxPooling1D())
  model_CNN.add(Dense(128, activation="relu"))
  model_CNN.add(Dropout(0.5))
  model_CNN.add(Flatten())
  model_CNN.add(Dense(1, activation='sigmoid'))

  # add checkpoint to save the network and stop if training doesn't improve MCTI
  checkpoint = keras.callbacks.ModelCheckpoint(save_as, monitor='val_accuracy', verbose=1, 
                              save_best_only=True, mode='max')
  callbacks_list = [checkpoint]

  model_CNN.compile(optimizer='adam', loss='binary_crossentropy', 
                        metrics=['accuracy', f1_m, precision_m, recall_m])

  # Fiting  the model
  history = model_CNN.fit(x_train, y_train, epochs=100, 
                        callbacks=callbacks_list, verbose=2, 
                        validation_data=(x_test, y_test), 
                        batch_size=16)

  # Evaluate the model
  [modelloss, modelaccuracy, 
  modelf1, modelprecision, 
  modelrecall] = model_CNN.evaluate(x_test, y_test, verbose=0)

  #plot_history(history)
  return modelaccuracy, (x_test, y_test)

In [ ]:
def train_network_lstm(save_as='best weights.h5'):
  
  # Creating the Network
  model_LSTM = Sequential()
  model_LSTM.add(Input(shape=np.shape(x_train[0])))
  model_LSTM.add(LSTM(64))
  model_LSTM.add(Dense(64, activation='relu'))
  model_LSTM.add(Flatten())
  model_LSTM.add(Dense(1, activation='sigmoid'))

  # add checkpoint to save the network and stop if training doesn't improve MCTI
  checkpoint = keras.callbacks.ModelCheckpoint(save_as, monitor='val_accuracy', verbose=1, 
                              save_best_only=True, mode='max')
  callbacks_list = [checkpoint]

  model_LSTM.compile(optimizer='adam', loss='binary_crossentropy', 
                        metrics=['accuracy', f1_m, precision_m, recall_m])

  # Fiting  the model
  history = model_LSTM.fit(x_train, y_train, epochs=100, 
                        callbacks=callbacks_list, verbose=2, 
                        validation_data=(x_test, y_test), 
                        batch_size=16)

  # Evaluate the model
  [modelloss, modelaccuracy, 
  modelf1, modelprecision, 
  modelrecall] = model_LSTM.evaluate(x_test, y_test, verbose=0)

  #plot_history(history)
  return modelaccuracy, (x_test, y_test)

# Training the networks

# NN

In [ ]:
i = 0
best_i = 0
best_acc_nn = 0

In [ ]:
test_data = []
print("STARTING TRAINING #" + str(i))
current_acc_nn, test_data = train_network_nn(
                          "/content/drive/MyDrive/Mestrado/PesosLDeep/best_weights_nn-" + str(i) + ".h5")
if current_acc_nn > best_acc_nn:
  best_acc_nn = current_acc_nn
  best_i = i


i+=1

In [ ]:
print("Best dnn data preprocessing Acc Score: " + str(best_acc_nn))
print(best_i)

In [ ]:
path = "/content/drive/MyDrive/Mestrado/PesosLDeep/best_weights_dnn-"+ str(best_i) + ".h5"

reconstructed_model_DNN = keras.models.load_model(path, 
                                                 custom_objects={'f1_m':f1_m, 
                                                                 "precision_m":precision_m, 
                                                                 "recall_m":recall_m})
# evaluate the model
loss, accuracy, f1_score, precision, recall = reconstructed_model_DNN.evaluate(test_data[0], 
                                                                              test_data[1], 
                                                                              verbose=0)

print('Base:')
print('Accuracy DNN: %f' % (accuracy*100))
print('f1_score DNN: %f' % (f1_score*100))
print('precision DNN: %f' % (precision*100))
print('recall DNN: %f' % (recall*100))

## DNN

In [ ]:
i = 0
best_i = 0
best_acc_dnn = 0
test_data = []
while i < 4:
  print("STARTING TRAINING #" + str(i))
  current_acc_dnn, test_data = train_network_dnn(
                              "/content/drive/MyDrive/Mestrado/PesosLDeep/best_weights_dnn-" + str(i) + ".h5")
  if current_acc_dnn > best_acc_dnn:
    best_acc_dnn = current_acc_dnn
    best_i = i
  i+=1
print("Best dnn data preprocessing Acc Score: " + str(best_acc_dnn))
print(best_i)

STARTING TRAINING #0
Epoch 1/100

Epoch 1: val_accuracy improved from -inf to 0.82258, saving model to /content/drive/MyDrive/Mestrado/PesosLDeep/best_weights_dnn-0.h5
93/93 - 83s - loss: 1.3411 - accuracy: 0.5539 - f1_m: 0.1635 - precision_m: 0.1383 - recall_m: 0.2688 - val_loss: 0.5386 - val_accuracy: 0.8226 - val_f1_m: 0.6360 - val_precision_m: 0.7847 - val_recall_m: 0.5604 - 83s/epoch - 893ms/step
Epoch 2/100

Epoch 2: val_accuracy did not improve from 0.82258
93/93 - 81s - loss: 0.4777 - accuracy: 0.7911 - f1_m: 0.6453 - precision_m: 0.7414 - recall_m: 0.6372 - val_loss: 0.7878 - val_accuracy: 0.6828 - val_f1_m: 0.2285 - val_precision_m: 0.4167 - val_recall_m: 0.1639 - 81s/epoch - 867ms/step
Epoch 3/100

Epoch 3: val_accuracy did not improve from 0.82258
93/93 - 80s - loss: 0.4020 - accuracy: 0.8369 - f1_m: 0.6905 - precision_m: 0.7763 - recall_m: 0.6781 - val_loss: 0.4657 - val_accuracy: 0.7849 - val_f1_m: 0.5607 - val_precision_m: 0.7778 - val_recall_m: 0.4736 - 80s/epoch - 865m

In [ ]:
path = "/content/drive/MyDrive/Mestrado/PesosLDeep/best_weights_dnn-"+ str(best_i) + ".h5"

reconstructed_model_DNN = keras.models.load_model(path, 
                                                 custom_objects={'f1_m':f1_m, 
                                                                 "precision_m":precision_m, 
                                                                 "recall_m":recall_m})
# evaluate the model
loss, accuracy, f1_score, precision, recall = reconstructed_model_DNN.evaluate(test_data[0], 
                                                                              test_data[1], 
                                                                              verbose=0)

print('Base:')
print('Accuracy DNN: %f' % (accuracy*100))
print('f1_score DNN: %f' % (f1_score*100))
print('precision DNN: %f' % (precision*100))
print('recall DNN: %f' % (recall*100))

Base:
Accuracy DNN: 89.784944
f1_score DNN: 84.410125
precision DNN: 92.567152
recall DNN: 77.808303


## CNN

In [ ]:
i = 0
best_i = 0
best_acc_cnn = 0
test_data = []
while i < 20:
  print("STARTING TRAINING #" + str(i))
  current_acc_cnn, test_data = train_network_cnn(
                              "/content/drive/MyDrive/Mestrado/PesosLDeep/best_weights_cnn-" + str(i) + ".h5")
  break
  if current_acc_cnn > best_acc_cnn:
    best_acc_cnn = current_acc_cnn
    best_i = i
  i+=1
print("Best cnn data preprocessing Acc Score: " + str(best_acc_cnn))
print(best_i)

STARTING TRAINING #0
Epoch 1/100

Epoch 1: val_accuracy improved from -inf to 0.61290, saving model to /content/drive/MyDrive/Mestrado/PesosLDeep/best_weights_cnn-0.h5
47/47 - 55s - loss: 0.7595 - accuracy: 0.5809 - f1_m: 0.2297 - precision_m: 0.2892 - recall_m: 0.2494 - val_loss: 0.6474 - val_accuracy: 0.6129 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - 55s/epoch - 1s/step
Epoch 2/100

Epoch 2: val_accuracy improved from 0.61290 to 0.75806, saving model to /content/drive/MyDrive/Mestrado/PesosLDeep/best_weights_cnn-0.h5
47/47 - 54s - loss: 0.6236 - accuracy: 0.6334 - f1_m: 0.2497 - precision_m: 0.3923 - recall_m: 0.2223 - val_loss: 0.5028 - val_accuracy: 0.7581 - val_f1_m: 0.5797 - val_precision_m: 0.8861 - val_recall_m: 0.4423 - 54s/epoch - 1s/step
Epoch 3/100

Epoch 3: val_accuracy improved from 0.75806 to 0.85484, saving model to /content/drive/MyDrive/Mestrado/PesosLDeep/best_weights_cnn-0.h5
47/47 - 53s - loss: 0.5260 - accuracy: 0.7534 - f1_m

KeyboardInterrupt: ignored

In [ ]:
path = "/content/drive/MyDrive/Mestrado/PesosLDeep/best_weights_cnn-"+ str(best_i) + ".h5"

reconstructed_model_cnn = keras.models.load_model(path, 
                                                 custom_objects={'f1_m':f1_m, 
                                                                 "precision_m":precision_m, 
                                                                 "recall_m":recall_m})
# evaluate the model
loss, accuracy, f1_score, precision, recall = reconstructed_model_cnn.evaluate(test_data[0], 
                                                                              test_data[1], 
                                                                              verbose=0)

print('Base:')
print('Accuracy cnn: %f' % (accuracy*100))
print('f1_score cnn: %f' % (f1_score*100))
print('precision cnn: %f' % (precision*100))
print('recall cnn: %f' % (recall*100))

Base:
Accuracy cnn: 94.086021
f1_score cnn: 90.685892
precision cnn: 100.000000
recall cnn: 83.414727


## LSTM

In [ ]:
i = 0
best_i = 0
best_acc_lstm = 0
test_data = []
while i < 3:
  print("STARTING TRAINING #" + str(i))
  current_acc_lstm, test_data = train_network_lstm(
                              "/content/drive/MyDrive/Mestrado/PesosLDeep/best_weights_lstm-" + str(i) + ".h5")
  if current_acc_lstm > best_acc_lstm:
    best_acc_lstm = current_acc_lstm
    best_i = i
  i+=1
print("Best lstm data preprocessing Acc Score: " + str(best_acc_lstm))
print(best_i)

Epoch 1/100

Epoch 1: val_accuracy improved from -inf to 0.61290, saving model to best weights LSTM.h5
47/47 - 141s - loss: 0.6766 - accuracy: 0.6213 - f1_m: 0.0153 - precision_m: 0.0120 - recall_m: 0.0213 - val_loss: 0.6722 - val_accuracy: 0.6129 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - 141s/epoch - 3s/step
Epoch 2/100

Epoch 2: val_accuracy did not improve from 0.61290
47/47 - 135s - loss: 0.6706 - accuracy: 0.6186 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 0.6674 - val_accuracy: 0.6129 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - 135s/epoch - 3s/step
Epoch 3/100

Epoch 3: val_accuracy did not improve from 0.61290
47/47 - 135s - loss: 0.6709 - accuracy: 0.6186 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 0.6718 - val_accuracy: 0.6129 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - 135s/epoch - 3s/step
E

KeyboardInterrupt: ignored

In [ ]:
path = "/content/drive/MyDrive/Mestrado/PesosLDeep/best_weights_lstm-"+ str(best_i) + ".h5"

reconstructed_model_LSTM = keras.models.load_model(path, 
                                                   custom_objects={'f1_m':f1_m, 
                                                                   "precision_m":precision_m, 
                                                                   "recall_m":recall_m})
# evaluate the model
loss, accuracy, f1_score, precision, recall = reconstructed_model_LSTM.evaluate(test_data[0], 
                                                                              test_data[1], 
                                                                     verbose=0)
print('Accuracy LSTM: %f' % (accuracy*100))
print('f1_score LSTM: %f' % (f1_score*100))
print('precision LSTM: %f' % (precision*100))
print('recall LSTM: %f' % (recall*100))

Accuracy LSTM: 61.290324
f1_score LSTM: 0.000000
precision LSTM: 0.000000
recall LSTM: 0.000000
